In [ ]:
import os
import pandas
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import ticker
from IPython.display import display, HTML
os.chdir('/home/idies/workspace/Temporary/raddick/cra_scratch/')
os.getcwd()

In [ ]:
baltimore_originators_df = pandas.read_csv('loans_processed.csv', encoding='utf-8', index_col='rownumber')
#baltimore_df.sample(3).T
print('done')

In [ ]:
print(len(baltimore_originators_df))
baltimore_originators_df = baltimore_originators_df[
    (baltimore_originators_df['state'] == 24) &
    (baltimore_originators_df['county'] == 510) &
    (baltimore_originators_df['loan_type'] == 4) &
    (baltimore_originators_df['action_taken_type'] == 1)
]
print(len(baltimore_originators_df))

baltimore_originators_df_bk = baltimore_originators_df

baltimore_originators_df.sample(3).T

In [ ]:
baltimore_tracts_df = pandas.read_csv('tracts_processed.csv', encoding='utf-8', index_col='rownumber')
print(len(baltimore_tracts_df))

baltimore_tracts_df = baltimore_tracts_df[(baltimore_tracts_df['state'] == 24) & (baltimore_tracts_df['county'] == 510)]
print(len(baltimore_tracts_df))

In [ ]:
neighborhoods_df = pandas.read_csv('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/census_tract_to_neighborhood.csv')
#baltimore_tracts_df = baltimore_tracts_df.
neighborhoods_df = neighborhoods_df.set_index('NAME10')

baltimore_tracts_df = baltimore_tracts_df.join(neighborhoods_df['CSA2010'], how='left', on='census_tract')
baltimore_tracts_df = baltimore_tracts_df.rename(columns={'CSA2010': 'neighborhood'})

baltimore_tracts_df_bk = baltimore_tracts_df

baltimore_tracts_df.sample(3)

In [ ]:
print('Reading from backup...')
baltimore_originators_df = baltimore_originators_df_bk

print('Totaling loan numbers and amounts...')
baltimore_originators_df = baltimore_originators_df.assign(nLoansTotal=baltimore_originators_df['nLoans1']+baltimore_originators_df['nLoans100k']+baltimore_originators_df['nLoans250k'])
baltimore_originators_df = baltimore_originators_df.assign(amtLoansTotal=baltimore_originators_df['amtLoans1']+baltimore_originators_df['amtLoans100k']+baltimore_originators_df['amtLoans250k'])
baltimore_originators_df = baltimore_originators_df.assign(amtLoansTotal_adjusted=baltimore_originators_df['amtLoans1_adjusted']+baltimore_originators_df['amtLoans100k_adjusted']+baltimore_originators_df['amtLoans250k_adjusted'])

print('Converting full level system to low/moderate/medium/upper...')
baltimore_originators_df = baltimore_originators_df.assign(cra_level = '')
baltimore_originators_df.loc[(baltimore_originators_df['income_group_total'].apply(lambda x: (x >= 1) & (x <=5))), 'cra_level'] = 'low'
baltimore_originators_df.loc[(baltimore_originators_df['income_group_total'].apply(lambda x: (x >= 6) & (x <=8))), 'cra_level'] = 'moderate'
baltimore_originators_df.loc[(baltimore_originators_df['income_group_total'].apply(lambda x: (x >= 9) & (x <=12))), 'cra_level'] = 'middle'
baltimore_originators_df.loc[(baltimore_originators_df['income_group_total'].apply(lambda x: (x == 13))), 'cra_level'] = 'upper'
baltimore_originators_df.loc[(baltimore_originators_df['income_group_total'].apply(lambda x: (x >= 14) & (x <=15))), 'cra_level'] = 'unknown'

#baltimore_originators_df.sample(3).T

baltimore_originators_df.groupby('cra_level').size().reindex(['low', 'moderate', 'middle', 'upper', 'unknown'])

In [ ]:
inst_year_df = baltimore_originators_df.groupby(['institution_name', 'activity_year'])['amtLoansTotal'].sum().unstack(1)#.sort_values)

sortorder = baltimore_originators_df[(baltimore_originators_df['institution_name'] != 'Unknown') & (baltimore_originators_df['activity_year'] == 2016)].groupby('institution_name', as_index=False)['amtLoansTotal'].sum().sort_values(by='amtLoansTotal', ascending=False)['institution_name'].values.tolist()
sortindex = pandas.Index(sortorder)
inst_year_df = inst_year_df.reindex(sortindex)

htmlstring = '<h1>Lending in Baltimore City by institution, 2009-2016</h1>'
htmlstring += '<table>'
htmlstring += '<tr>'
htmlstring += '<th>Institution</th>'
for i in range(2009, 2017):
    htmlstring += '<th>'+str(i)+'</th>'
htmlstring += '</tr>'
for idx, thisrow in inst_year_df.iterrows():
    htmlstring += '<tr>'
    htmlstring += '<td>{0:}</td>'.format(idx)
    for i in range(5, 13):
        #if (thisrow.iloc[i] == np.nan):
        if ('<td>${0:,.0f}</td>'.format(thisrow.iloc[i]) == '<td>$nan</td>'):
            htmlstring += '<td></td>'
        else:
            htmlstring += '<td>${0:,.0f}</td>'.format(thisrow.iloc[i])
    htmlstring += '</tr>'
htmlstring += '</table>'

display(HTML(htmlstring))


In [ ]:
inst_year_df = baltimore_originators_df.groupby(['institution_name', 'activity_year'])['amtLoansTotal_adjusted'].sum().unstack(1)#.sort_values)

sortorder = baltimore_originators_df[(baltimore_originators_df['institution_name'] != 'Unknown') & (baltimore_originators_df['activity_year'] == 2016)].groupby('institution_name', as_index=False)['amtLoansTotal'].sum().sort_values(by='amtLoansTotal', ascending=False)['institution_name'].values.tolist()
sortindex = pandas.Index(sortorder)
inst_year_df = inst_year_df.reindex(sortindex)

htmlstring = '<h1>Lending in Baltimore City by institution, 2004-2008 (inflation-adjusted to 2016 dollars)</h1>'
htmlstring += '<table>'
htmlstring += '<tr>'
htmlstring += '<th>Institution</th>'
for i in range(2004, 2009):
    htmlstring += '<th>'+str(i)+'</th>'
htmlstring += '</tr>'
for idx, thisrow in inst_year_df.iterrows():
    htmlstring += '<tr>'
    htmlstring += '<td>{0:}</td>'.format(idx)
    for i in range(0, 5):
        #if (thisrow.iloc[i] == np.nan):
        if ('<td>${0:,.0f}</td>'.format(thisrow.iloc[i]) == '<td>$nan</td>'):
            htmlstring += '<td></td>'
        else:
            htmlstring += '<td>${0:,.0f}</td>'.format(thisrow.iloc[i])
    htmlstring += '</tr>'
htmlstring += '</table>'

display(HTML(htmlstring))


In [ ]:
def ymillions(x,pos):
    return '{0:.0f}'.format(x/1000000)

year_df = baltimore_originators_df.groupby(['activity_year'])['amtLoansTotal'].sum()#.unstack(1)
year_adj_df = baltimore_originators_df.groupby(['activity_year'])['amtLoansTotal_adjusted'].sum()#.unstack(1)

#sortorder = baltimore_originators_df[(baltimore_originators_df['institution_name'] != 'Unknown') & (baltimore_originators_df['activity_year'] == 2016)].groupby('institution_name', as_index=False)['amtLoansTotal'].sum().sort_values(by='amtLoansTotal', ascending=False)['institution_name'].values.tolist()
#sortindex = pandas.Index(sortorder)
#inst_year_df = inst_year_df.reindex(sortindex)

fig, ax1 = plt.subplots(figsize=(12,6))

ax1.plot(year_df.index, year_df.values, marker='o', label='Real')
ax1.plot(year_adj_df.index, year_adj_df.values, marker='o', label='Inflation-adjusted to 2016')

ax1.xaxis.set_major_locator(ticker.FixedLocator([2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]))

ax1.set_ylim((0,600000000))
ax1.yaxis.set_major_locator(ticker.AutoLocator())
ax1.yaxis.set_major_formatter(ticker.FuncFormatter(ymillions))

plt.tick_params(axis='both', which='major', labelsize=12)
plt.xlabel('Year', fontsize=16)
plt.ylabel('Total loans (millions)', fontsize=16)
plt.title('Total CRA loans in Baltimore City', fontsize=18)
plt.legend(fontsize=12)
plt.show()